In [126]:
#SELEN ERDOGAN - 1901022038

In [127]:
import numpy as np
import pandas as pd

In [128]:
df = pd.read_csv('spam_ham_dataset.csv',encoding='ISO-8859-1')
df

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [129]:
columns_to_drop = ["Unnamed: 0", "label_num"]
for col in columns_to_drop:
    if col in df.columns:
        df.drop(col, axis=1, inplace=True)

In [130]:
df.columns = ['labels','content']
df

,labels,content
0,ham,Subject: enron methanol ; meter # : 988291\r\n...
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see..."
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar..."
3,spam,"Subject: photoshop , windows , office . cheap ..."
4,ham,Subject: re : indian springs\r\nthis deal is t...
...,...,...
5166,ham,Subject: put the 10 on the ft\r\nthe transport...
5167,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...
5168,ham,Subject: calpine daily gas nomination\r\n>\r\n...
5169,ham,Subject: industrial worksheets for august 2000...


In [131]:
df['b_labels'] = df['labels'].map({'ham': 0, 'spam': 1})
df

,labels,content,b_labels
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...
5166,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,ham,Subject: industrial worksheets for august 2000...,0


In [132]:
training_test_index = round(len(df) * 0.75)

training_set = df[:training_test_index].reset_index(drop=True)
test_set = df[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(3878, 3)
(1293, 3)


In [133]:
training_set['content'] = training_set['content'].str.replace('\W', ' ', regex=True)
training_set['content'] = training_set['content'].str.lower()
training_set['content'] = training_set['content'].str.replace('subject', '', regex=False)
training_set['content'] = training_set['content'].str.replace('and', '', regex=False)
training_set['content'] = training_set['content'].str.replace('or', '', regex=False)
training_set['content'] = training_set['content'].str.replace('an', '', regex=False)
training_set['content'] = training_set['content'].str.replace('a', '', regex=False)
training_set['content'] = training_set['content'].str.replace('to', '', regex=False)
training_set['content'] = training_set['content'].str.replace('from', '', regex=False)
training_set['content'] = training_set['content'].str.replace('in', '', regex=False)
training_set['content'] = training_set['content'].str.replace('for', '', regex=False)
training_set['content'] = training_set['content'].str.replace('on', '', regex=False)
training_set['content'] = training_set['content'].str.replace('at', '', regex=False)

training_set

,labels,content,b_labels
0,ham,enr methol meter 988291 this is foll...,0
1,ham,hpl nom f jury 9 2001 see ttched file ...,0
2,ham,ne retret ho ho ho we re round tht mos...,0
3,spam,phoshop wdows office chep m trendg ...,1
4,ham,re di sprgs this del is book the teco pv...,0
...,...,...,...
3873,ham,re crecti nom f 4 30 01 we gree e...,0
3874,ham,re tensk iv 1 01 txu revised their rtes...,0
3875,spam,you gott check out h g hme dietry therpy ...,1
3876,ham,re june 00 producti o cn hewitt ltd ...,0


In [134]:
training_set['content'] = training_set['content'].str.split()

words = []
for text in training_set['content']:
    for word in text:
        words.append(word)
words = list(set(words))

len(words) 

39972

In [135]:
word_counts_per_text = {unique_word: [0] * len(training_set['content']) for unique_word in words}

for index, data in enumerate(training_set['content']):
    for word in data:
        word_counts_per_text[word][index] +=1

In [136]:
word_counts = pd.DataFrame(word_counts_per_text)

In [137]:
# Concatenate training set df and words df
training_set_clean = pd.concat([training_set, word_counts],axis=1)
training_set_clean

,labels,content,b_labels,svee,wcdfow,pek,fmtv,ychtmster,celebrte,mfgtg,...,pjgfewift,158625,druggg,sfd,933,vegetti,populrity,365,stite,mcbl
0,ham,"[enr, methol, meter, 988291, this, is, follow,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[hpl, nom, f, jury, 9, 2001, see, ttched, file...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[ne, retret, ho, ho, ho, we, re, round, tht, m...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,spam,"[phoshop, wdows, office, chep, m, trendg, bsem...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[re, di, sprgs, this, del, is, book, the, teco...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3873,ham,"[re, crecti, nom, f, 4, 30, 01, we, gree, eile...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3874,ham,"[re, tensk, iv, 1, 01, txu, revised, their, rt...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3875,spam,"[you, gott, check, out, h, g, hme, dietry, the...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3876,ham,"[re, june, 00, producti, o, cn, hewitt, ltd, f...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [138]:
spam_mail = training_set_clean[training_set_clean['labels'] == 'spam']
ham_mail = training_set_clean[training_set_clean['labels'] == 'ham']

p_spam = len(spam_mail) / len(training_set_clean)
p_ham = len(ham_mail) / len(training_set_clean)

n_words_per_spam_mail = spam_mail['content'].apply(len)
n_spam = n_words_per_spam_mail.sum()

n_words_per_ham_mail = ham_mail['content'].apply(len)
n_ham = n_words_per_ham_mail.sum()

n_words = len(words)

alpha = 1

In [139]:
parameters_spam = {unique_word: 0 for unique_word in words}
parameters_ham = {unique_word: 0 for unique_word in words}

In [140]:
for word in words:
    n_word_given_spam = spam_mail[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_words)
    parameters_spam[word] = p_word_given_spam

    n_word_given_ham = ham_mail[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_words)
    parameters_ham[word] = p_word_given_ham

In [141]:
def classifyingTest(mail):
   
    mail = re.sub('\W', ' ', mail)
    mail = mail.lower().split()

    p_spam_given_mail = p_spam
    p_ham_given_mail = p_ham

    for word in mail:
        if word in parameters_spam:
            p_spam_given_mail *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_mail *= parameters_ham[word]

    if p_ham_given_mail >= p_spam_given_mail:
        return 'ham'
    elif p_spam_given_mail > p_ham_given_mail:
        return 'spam'

test_set['predicted'] = test_set['content'].apply(classifyingTest)
test_set 

,labels,content,b_labels,predicted
0,ham,"Subject: enron / hpl actuals - nov . 7 , 2000\...",0,ham
1,spam,Subject: exclusive positions in montanayoocwo\...,1,spam
2,ham,Subject: natural gas nomination for december 2...,0,ham
3,ham,Subject: imperial sugar ' s volumes will be 14...,0,ham
4,ham,Subject: re : first delivery - wagner oil\r\nv...,0,ham
...,...,...,...,...
1288,ham,Subject: put the 10 on the ft\r\nthe transport...,0,ham
1289,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0,ham
1290,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0,ham
1291,ham,Subject: industrial worksheets for august 2000...,0,ham


In [142]:
true_positives = 0
false_positives = 0
false_negatives = 0

for row in test_set.iterrows():
    row = row[1]
    if row['labels'] == row['predicted']:
        true_positives += 1
    elif row['labels'] == 'spam' and row['predicted'] == 'ham':
        false_negatives += 1
    elif row['labels'] == 'ham' and row['predicted'] == 'spam':
        false_positives += 1

precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1_score = 2 * (precision * recall) / (precision + recall)

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct / total)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1_score)


Correct: 1164
Incorrect: 129
Accuracy: 0.9002320185614849
Precision: 0.9965753424657534
Recall: 0.9030256012412723
F1 Score: 0.9474969474969475
